# Examples of the same mission at different UI levels

Here we present a sort of [Rosetta Stone](https://en.wikipedia.org/wiki/Rosetta_Stone) for the same mission at different UI levels.
This is to show the code used to make and solve similar mission optimization problems at different UI levels.

In each of these cases we'll set the maximum iterations to 0 to save execution time.
If you wanted to actually solve these problems, you'd need to set the maximum iterations to a larger number, like 50.

```{note}
Because we're setting the max iter to 0, we see an "Optimization FAILED" output, but do not be concerned, this is expected as we're simply showing the different levels here, not actually solving the problems.
```

## Level 1

This is the most basic level of the UI.
There's a command-line interface (CLI) for Level 1, or you can use the Python API shown below.
If you want to make minor modifications to `phase_info`, you can do so here, but Level 1 largely assumes you're using the default setup.

In [ ]:
import aviary.api as av

prob = av.run_aviary('models/test_aircraft/aircraft_for_bench_FwFm.csv',
                     av.default_height_energy_phase_info,
                     max_iter=0, optimizer="SLSQP", make_plots=False)

## Level 2

This level grants more flexibility both in defining the `phase_info` object but also in calling the individual methods of `AviaryProblem` when setting up and running your model.
You can modify the methods you call, what they do, and what info they're given here.
This is much more verbose than Level 1.
In the absence of additional arguments to the methods, much of the default behavior here is the same as Level 1.

In [ ]:
import aviary.api as av

phase_info = {
    "pre_mission": {"include_takeoff": False, "optimize_mass": True},
    "climb": {
        "subsystem_options": {"core_aerodynamics": {"method": "computed"}},
        "user_options": {
            "optimize_mach": False,
            "optimize_altitude": False,
            "polynomial_control_order": 1,
            "num_segments": 5,
            "order": 3,
            "solve_for_distance": False,
            "initial_mach": (0.2, "unitless"),
            "final_mach": (0.72, "unitless"),
            "mach_bounds": ((0.18, 0.74), "unitless"),
            "initial_altitude": (0.0, "ft"),
            "final_altitude": (32000.0, "ft"),
            "altitude_bounds": ((0.0, 34000.0), "ft"),
            "throttle_enforcement": "path_constraint",
            "fix_initial": True,
            "constrain_final": False,
            "fix_duration": False,
            "initial_bounds": ((0.0, 0.0), "min"),
            "duration_bounds": ((64.0, 192.0), "min"),
            "add_initial_mass_constraint": False,
        },
        "initial_guesses": {"times": ([0, 128], "min")},
    },
    "cruise": {
        "subsystem_options": {"core_aerodynamics": {"method": "computed"}},
        "user_options": {
            "optimize_mach": False,
            "optimize_altitude": False,
            "polynomial_control_order": 1,
            "num_segments": 5,
            "order": 3,
            "solve_for_distance": False,
            "initial_mach": (0.72, "unitless"),
            "final_mach": (0.72, "unitless"),
            "mach_bounds": ((0.7, 0.74), "unitless"),
            "initial_altitude": (32000.0, "ft"),
            "final_altitude": (34000.0, "ft"),
            "altitude_bounds": ((23000.0, 38000.0), "ft"),
            "throttle_enforcement": "boundary_constraint",
            "fix_initial": False,
            "constrain_final": False,
            "fix_duration": False,
            "initial_bounds": ((64.0, 192.0), "min"),
            "duration_bounds": ((56.5, 169.5), "min"),
        },
        "initial_guesses": {"times": ([128, 113], "min")},
    },
    "descent": {
        "subsystem_options": {"core_aerodynamics": {"method": "computed"}},
        "user_options": {
            "optimize_mach": False,
            "optimize_altitude": False,
            "polynomial_control_order": 1,
            "num_segments": 5,
            "order": 3,
            "solve_for_distance": False,
            "initial_mach": (0.72, "unitless"),
            "final_mach": (0.36, "unitless"),
            "mach_bounds": ((0.34, 0.74), "unitless"),
            "initial_altitude": (34000.0, "ft"),
            "final_altitude": (500.0, "ft"),
            "altitude_bounds": ((0.0, 38000.0), "ft"),
            "throttle_enforcement": "path_constraint",
            "fix_initial": False,
            "constrain_final": True,
            "fix_duration": False,
            "initial_bounds": ((120.5, 361.5), "min"),
            "duration_bounds": ((29.0, 87.0), "min"),
        },
        "initial_guesses": {"times": ([241, 58], "min")},
    },
    "post_mission": {
        "include_landing": False,
        "constrain_range": True,
        "target_range": (1906, "nmi"),
    },
}

csv_path = "models/test_aircraft/aircraft_for_bench_FwFm.csv"

prob = av.AviaryProblem()

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs(csv_path, phase_info)

prob.check_and_preprocess_inputs()

prob.add_pre_mission_systems()

prob.add_phases()

prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

prob.add_driver("SLSQP", max_iter=0)

prob.add_design_variables()

prob.add_objective(objective_type="mass", ref=-1e5)

prob.setup()

prob.set_initial_guesses()

prob.run_aviary_problem(record_filename='level2_example.db', suppress_solver_print=True, make_plots=False)

## Level 3

This level is the most flexible and the code will be the most verbose.
This directly calls the OpenMDAO and Dymos methods used to set up and run the model.
You have supreme control over every part of the model and mission definition, but that flexibility results in much more code.

In [ ]:
import dymos as dm
import openmdao.api as om
import scipy.constants as _units

import aviary.api as av


prob = om.Problem(model=om.Group())
driver = prob.driver = om.ScipyOptimizeDriver()
driver.options["optimizer"] = "SLSQP"
driver.options["maxiter"] = 1

##########################################
# Aircraft Input Variables and Options   #
##########################################

csv_path = "models/test_aircraft/aircraft_for_bench_FwFm.csv"

aviary_inputs, _ = av.create_vehicle('models/test_aircraft/aircraft_for_bench_FwFm.csv')

engine = av.EngineDeck(options=aviary_inputs)
av.preprocess_propulsion(aviary_inputs, [engine])

alt_airport = 0  # ft

alt_i_climb = 0*_units.foot  # m
alt_f_climb = 32000.0*_units.foot  # m
mass_i_climb = 131000*_units.lb  # kg
mass_f_climb = 126000*_units.lb  # kg
mach_i_climb = 0.2
mach_f_climb = 0.72
distance_i_climb = 0*_units.nautical_mile  # m
distance_f_climb = 160.3*_units.nautical_mile  # m
t_i_climb = 0. * _units.minute  # sec
t_f_climb = 128 * _units.minute  # sec
t_duration_climb = t_f_climb - t_i_climb

alt_i_cruise = 32000*_units.foot  # m
alt_f_cruise = 34000*_units.foot  # m
alt_min_cruise = 32000*_units.foot  # m
alt_max_cruise = 34000*_units.foot  # m
mass_i_cruise = 126000*_units.lb  # kg
mass_f_cruise = 102000*_units.lb  # kg
cruise_mach = 0.79
distance_i_cruise = 160.3*_units.nautical_mile  # m
distance_f_cruise = 3243.9*_units.nautical_mile  # m
t_i_cruise = 128. * _units.minute  # sec
t_f_cruise = 241. * _units.minute  # sec
t_duration_cruise = t_f_cruise - t_i_cruise

alt_i_descent = 34000*_units.foot
alt_f_descent = 500*_units.foot
mach_i_descent = 0.72
mach_f_descent = 0.3
mass_i_descent = 102000*_units.pound
mass_f_descent = 101000*_units.pound
distance_i_descent = 3243.9*_units.nautical_mile
distance_f_descent = 3378.7*_units.nautical_mile
t_i_descent = 241. * _units.minute
t_f_descent = 299. * _units.minute
t_duration_descent = t_f_descent - t_i_descent

##########################
# Design Variables       #
##########################

# Nudge it a bit off the correct answer to verify that the optimize takes us there.
aviary_inputs.set_val(av.Mission.Design.GROSS_MASS, 135000.0, units='lbm')

prob.model.add_design_var(av.Mission.Design.GROSS_MASS, units='lbm',
                            lower=100000.0, upper=200000.0, ref=135000)

##################
# Define Phases  #
##################
num_segments_climb = 6
num_segments_cruise = 1
num_segments_descent = 5

climb_seg_ends, _ = dm.utils.lgl.lgl(num_segments_climb + 1)
descent_seg_ends, _ = dm.utils.lgl.lgl(num_segments_descent + 1)

transcription_climb = dm.Radau(
    num_segments=num_segments_climb, order=3, compressed=True,
    segment_ends=climb_seg_ends)
transcription_cruise = dm.Radau(
    num_segments=num_segments_cruise, order=3, compressed=True)
transcription_descent = dm.Radau(
    num_segments=num_segments_descent, order=3, compressed=True,
    segment_ends=descent_seg_ends)

climb_options = av.HeightEnergyPhaseBuilder(
    'test_climb',
    user_options=av.AviaryValues({
        'initial_altitude': (alt_i_climb, 'm'),
        'final_altitude': (alt_f_climb, 'm'),
        'initial_mach': (mach_i_climb, 'unitless'),
        'final_mach': (mach_f_climb, 'unitless'),
        'fix_initial': (True, 'unitless'),
        'use_polynomial_control': (False, 'unitless'),
    }),
    core_subsystems=av.default_mission_subsystems,
    subsystem_options={'core_aerodynamics': {'method': 'computed'}},
    transcription=transcription_climb,
)

cruise_options = av.HeightEnergyPhaseBuilder(
    'test_cruise',
    user_options=av.AviaryValues({
        'initial_altitude': (alt_min_cruise, 'm'),
        'final_altitude': (alt_max_cruise, 'm'),
        'initial_mach': (cruise_mach, 'unitless'),
        'final_mach': (cruise_mach, 'unitless'),
        'required_available_climb_rate': (300, 'ft/min'),
        'fix_initial': (False, 'unitless'),
    }),
    core_subsystems=av.default_mission_subsystems,
    subsystem_options={'core_aerodynamics': {'method': 'computed'}},
    transcription=transcription_cruise,
)

descent_options = av.HeightEnergyPhaseBuilder(
    'test_descent',
    user_options=av.AviaryValues({
        'final_altitude': (alt_f_descent, 'm'),
        'initial_altitude': (alt_i_descent, 'm'),
        'initial_mach': (mach_i_descent, 'unitless'),
        'final_mach': (mach_f_descent, 'unitless'),
        'fix_initial': (False, 'unitless'),
        'use_polynomial_control': (False, 'unitless'),
    }),
    core_subsystems=av.default_mission_subsystems,
    subsystem_options={'core_aerodynamics': {'method': 'computed'}},
    transcription=transcription_descent,
)

av.preprocess_crewpayload(aviary_inputs)

# Upstream static analysis for aero
prob.model.add_subsystem(
    'pre_mission',
    av.CorePreMission(aviary_options=aviary_inputs,
                    subsystems=av.default_premission_subsystems),
    promotes_inputs=['aircraft:*', 'mission:*'],
    promotes_outputs=['aircraft:*', 'mission:*'])

# directly connect phases (strong_couple = True), or use linkage constraints (weak
# coupling / strong_couple=False)
strong_couple = False

climb = climb_options.build_phase(aviary_options=aviary_inputs)

cruise = cruise_options.build_phase(aviary_options=aviary_inputs)

descent = descent_options.build_phase(aviary_options=aviary_inputs)

traj = prob.model.add_subsystem('traj', dm.Trajectory())

# if fix_initial is false, can we always set input_initial to be true for
# necessary states, and then ignore if we use a linkage?
climb.set_time_options(fix_initial=True, fix_duration=False, units='s',
                        duration_bounds=(t_duration_climb*0.5, t_duration_climb*2),
                        duration_ref=t_duration_climb)
cruise.set_time_options(fix_initial=False, fix_duration=False, units='s',
                        duration_bounds=(t_duration_cruise*0.5, t_duration_cruise*2),
                        duration_ref=t_duration_cruise,
                        initial_bounds=(t_duration_climb*0.5, t_duration_climb*2))
descent.set_time_options(
    fix_initial=False, fix_duration=False, units='s',
    duration_bounds=(t_duration_descent*0.5, t_duration_descent*2),
    duration_ref=t_duration_descent,
    initial_bounds=(
        (t_duration_cruise + t_duration_climb)*0.5,
        (t_duration_cruise + t_duration_climb)*2))

traj.add_phase('climb', climb)

traj.add_phase('cruise', cruise)

traj.add_phase('descent', descent)

traj.link_phases(["climb", "cruise", "descent"], [
                    "time", "mass", "distance"], connected=strong_couple)

traj = av.setup_trajectory_params(prob.model, traj, aviary_inputs)

##########################
# Constraints            #
##########################

ecomp = om.ExecComp('fuel_burned = initial_mass - descent_mass_final',
                    initial_mass={'units': 'lbm', 'shape': 1},
                    descent_mass_final={'units': 'lbm', 'shape': 1},
                    fuel_burned={'units': 'lbm', 'shape': 1})

prob.model.add_subsystem('fuel_burn', ecomp,
                            promotes_inputs=[
                                ('initial_mass', av.Mission.Design.GROSS_MASS)],
                            promotes_outputs=['fuel_burned'])

prob.model.connect("traj.descent.states:mass",
                    "fuel_burn.descent_mass_final", src_indices=[-1])

##########################
# Add Objective Function #
##########################

prob.model.add_objective("fuel_burned", ref=1e4)

# Set initial default values for all LEAPS aircraft variables.
av.set_aviary_initial_values(prob.model, aviary_inputs)

prob.model.add_subsystem(
    'input_sink',
    av.VariablesIn(aviary_options=aviary_inputs),
    promotes_inputs=['*'],
    promotes_outputs=['*']
)

prob.setup(force_alloc_complex=True)

###########################################
# Initial Settings for States and Controls #
###########################################

prob.set_val('traj.climb.t_initial', t_i_climb, units='s')
prob.set_val('traj.climb.t_duration', t_duration_climb, units='s')

prob.set_val('traj.climb.controls:altitude', climb.interp(
    av.Dynamic.Mission.ALTITUDE, ys=[alt_i_climb, alt_f_climb]), units='m')
prob.set_val(
    'traj.climb.controls:mach', climb.interp(
        av.Dynamic.Mission.MACH, ys=[mach_i_climb, mach_f_climb]), units='unitless')
prob.set_val('traj.climb.states:mass', climb.interp(
    av.Dynamic.Mission.MASS, ys=[mass_i_climb, mass_f_climb]), units='kg')
prob.set_val('traj.climb.states:distance', climb.interp(
    av.Dynamic.Mission.DISTANCE, ys=[distance_i_climb, distance_f_climb]), units='m')

prob.set_val('traj.cruise.t_initial', t_i_cruise, units='s')
prob.set_val('traj.cruise.t_duration', t_duration_cruise, units='s')

prob.set_val('traj.cruise.polynomial_controls:altitude', cruise.interp(
    av.Dynamic.Mission.ALTITUDE, ys=[alt_i_cruise, alt_f_cruise]), units='m')
prob.set_val(
    'traj.cruise.polynomial_controls:mach', cruise.interp(
        av.Dynamic.Mission.MACH, ys=[cruise_mach, cruise_mach]), units='unitless')
prob.set_val('traj.cruise.states:mass', cruise.interp(
    av.Dynamic.Mission.MASS, ys=[mass_i_cruise, mass_f_cruise]), units='kg')
prob.set_val('traj.cruise.states:distance', cruise.interp(
    av.Dynamic.Mission.DISTANCE, ys=[distance_i_cruise, distance_f_cruise]), units='m')

prob.set_val('traj.descent.t_initial', t_i_descent, units='s')
prob.set_val('traj.descent.t_duration', t_duration_descent, units='s')

prob.set_val('traj.descent.controls:altitude', descent.interp(
    av.Dynamic.Mission.ALTITUDE, ys=[alt_i_descent, alt_f_descent]), units='m')
prob.set_val(
    'traj.descent.controls:mach', descent.interp(
        av.Dynamic.Mission.MACH, ys=[mach_i_descent, mach_f_descent]), units='unitless')
prob.set_val('traj.descent.states:mass', descent.interp(
    av.Dynamic.Mission.MASS, ys=[mass_i_descent, mass_f_descent]), units='kg')
prob.set_val('traj.descent.states:distance', descent.interp(
    av.Dynamic.Mission.DISTANCE, ys=[distance_i_descent, distance_f_descent]), units='m')

# Turn off solver printing so that the SNOPT output is readable.
prob.set_solver_print(level=0)

dm.run_problem(prob, simulate=False, make_plots=False, simulate_kwargs={
                'times_per_seg': 100, 'atol': 1e-9, 'rtol': 1e-9})